In [ ]:
from functions import *

from math import pi
import warnings
warnings.filterwarnings('ignore')

# USER TRANSMITTER 

The first thing that we need to do is to load the file with the settings. These are all the parameters that we are going to use for the development of the project.

In [ ]:
f = open('settings.ini')
settings = json.load(f)
 
for i in settings:
    print(i, end=': ')
    print(settings[i])
    
f.close()

#the transmitted power is in W, while the transmitter and receiver gains are in dB

In [ ]:
#reading the file with the packets to transmit: if GOOD = 1, it's an ACK, if GOOD = 0 it's a NACK

acks = pd.read_csv("N_ACK.csv")
num_packets = acks.shape[0]
print(num_packets)
acks

In [ ]:
f = open('N_ACK.csv')
csv_file = csv.reader(f)
N_ACK = []
for row in csv_file:
        N_ACK.append(row)
        
print(N_ACK)

for i in range(len(N_ACK)-1):
    if(N_ACK[i+1][1]=='1'):
        msg=createMessageACK(settings['SYNCpattern'],settings['SV_PRN_ID'],int(N_ACK[i+1][0]))
    if(N_ACK[i+1][1]=='0'):
        msg=createMessageNACK(settings['SYNCpattern'],settings['SV_PRN_ID'],int(N_ACK[i+1][0]))
    print(msg)

### Loading the Galileo E1 PRNs
In order to generate the signal, we need first of all to load the pseudo random noise sequences.

There exist two different kinds of PRN codes for Galileo E1: E1B and E1C.
They are contained, in hexadecimal format, in the files called repectively "E1B - Gal.txt" and "E1C - Gal.txt"

For this simulation, we only consider the E1B PRNs:

In [ ]:
filename = "E1B - Gal.txt"

# defining the name of the column of the dataframe called e1b
columns = ["Hexadecimal"]

#reading the file
e1b = pd.read_table(filename, sep='\n', header=None, names=columns)

# deleting the first 15 characters (not part of the code)
e1b['Hexadecimal'] = [el[16::] for el in e1b['Hexadecimal']]

# creating a new column with the codes indexes and placing it as first column
e1b['Code index'] = np.arange(1,51,1)
column_list = ["Code index", "Hexadecimal"]
e1b = e1b.reindex(columns=column_list)

# printing the first lines of the dataframe
e1b.head()

In [ ]:
# last code (just to verify if the loading operation is correct)
print(e1b['Hexadecimal'][49])

In [ ]:
# creation of a new column of the dataframe with the binary representation of each PRN
e1b['Binary'] = e1b['Hexadecimal'].apply(hex2bin)

#creation of a copy of the dataframe to use without modifying the original dataframe
e1bcopy = e1b.copy()
e1b

In [ ]:
print(e1b['Binary'][49])    # to verify if the conversion was successful, we print the last code in binary:

# since the last code starts with 9 7 ... in hexadecimal, the binary representation should start with 
# 1001 0111 ...
# and since it ends with ... 2 0, the binary representation should end with
# ... 0010 0000

## MODULATION

The signal sent by Galileo is supposed to be modulated with a Binary Offset Carrier modulation. In particular, the one that is used for Galileo E1 signal is the BOC(1,1) modulation, that utilizes a Non-return to zero square subcarrier with a period Tp equal to the chip rate Tc.

In order to generate the BOC(1,1) signal we need to modify the codes. Indeed, the sequence to modulate with the BOC(1,1) square subcarrier is obtained by applying a XOR between the data message and the PRN. Alternatively, obtaining the same result, it is possible to convert the binary sequences (both the data message and the PRNs) by considering a -1 instead of a 1 and by considering a 1 instead of a 0.

So the sequence 0 1 1 1 0 becomes 1 -1 -1 -1 1 (this is valid for both the data message and the PRN). Then instead of applying a XOR we can simply multiply the data message and the PRN, the final result is the same.

Indeed, by converting 0 in 1 and 1 in -1 we get:

|  | BIT 1 | BIT 2 | XOR | CONVERSION |
| :-: | :-: | :-: | :-: | :-: |
| 1) | 0 | 0 | 0 | 1 |
| 2) | 0 | 1 | 1 | -1 |
| 3) | 1 | 0 | 1 | -1 |
| 4) | 1 | 1 | 0 | 1 |

that is the same we get with the product after the conversion:

| | BIT 1 | BIT 2 | PRODUCT 
| :-: | :-: | :-: | :-: |
| 1) | 1 | 1 | 1 |
| 2) | 1 | -1 | -1 |
| 3) | -1 | 1 | -1 |
| 4) | -1 | -1 | 1 |

Therefore now we create another dataframe where we store the modified PRNs.

In [ ]:
e1bmodified = e1b.copy()
e1bmodified['Modified'] = e1bmodified['Binary'].apply(conversion)

e1bmodifiedcopy = e1bmodified.copy()
e1bmodified

## Visualization of PRNs in time domain

Now that the PRNs have been loaded, it is possible to visualize the first samples of them. This is done by using two functions defined below.

In [ ]:
plot_prn_zeros(20, e1bcopy['Binary'][0], settings['chipRate'])

In [ ]:
plot_prn_modified(20, e1bmodifiedcopy['Modified'][0], settings['chipRate'])

## BOC(1,1) MODULATION

The BOC(1,1) modulation is obtained by multiplying 4 components:
- the data message we need to transmit (given by a sequence of symbols with symbol period Tb);
- the PRN sequence we use to spread the data message (given by a sequence of chips with chip period Tc);
- a square wave subcarrier with period Tp = Tc (due to n = m = 1)
- a sinusoidal carrier 

Since for this project the I/Q samples must be generated in baseband, the multiplication by the sinusoidal carrier is not needed.

In [ ]:
# HERE WE NEED TO INSERT OUR REAL MESSAGE TO TRANSMIT

In [ ]:
#first we generate a random binary message of 80 bits (so it contains only zeros and ones)
message_len = 80
message = np.random.randint(0,2,message_len)
        
print("Data message:\n", message)

In [ ]:
#Now we generate the BOC subcarrier, that is a square NRZ wave. The period of the waveform is given by Tc
#The needed length of the subcarrier is 654720. This will be clarified later, but basically the spreaded sequence
#is composed of 327360 symbols (data message is 80 symbols, and each symbol is spreaded with a PRN, that has 4092
#chips, so 80*4092 = 327360) and each symbol of the spreaded sequence is kept constant for 2 samples of the 
#subcarrier, therefore 327360*2 = 654720.

Rc = 1.023e06   # chip rate
Tc = 1 / Rc     # chip period 

rep = 2 
s = np.array([1, -1])
subcarrier = np.tile(s, 327360) # we repeat the values 1 and -1 for 327360 times in order to have an array of 654720 elements
                             

#now we plot the first 20 samples of the subcarrier
xticks = np.arange(0,10*Tc, Tc)
yticks = [-1, 0, 1]

t = np.arange(0, 10*Tc, Tc/2)
fig = plt.figure(figsize=(15,5))
ax = fig.add_subplot(1,1,1)
ax.step(t, subcarrier[0:20], where='post', lw=3)
ax.set_xticks(xticks)
ax.set_yticks(yticks)
ax.set_title('NRZ square subcarrier for BOC(1,1)', fontsize=15)
ax.set_xlabel('Time [s]', fontsize=15)
ax.set_ylabel('Subcarrier values', fontsize=15)
ax.grid()

print("Subcarrier length:", len(subcarrier))  
print(subcarrier)

In [ ]:
#now we have the data message, the subcarrier and the prn can be retrieved from the dataframe
#we can create a function that does the modulation and (if flag = True) plots the signals

modulated = boc(message, subcarrier, settings["bitRate"], settings["chipRate"], settings['SV_PRN_ID'], np.array(e1bmodifiedcopy['Modified']), True)
print("\nModulated message:\n", modulated)

# SAMPLING OF THE SIGNAL

Before transmitting the signal we can sample it, since this will be useful also at the receiver side, where we will compute the I/Q samples of the received signal.
The sampling frequencies used in these cases are in the order of 4 or 5 MHz. A good choice might be to consider a sampling frequency $F_s = 4.092$ MHz (with a sampling period $T_s = \frac{1}{F_s} = 2.4438\,\cdot10^{-7} = \frac{T_c}{4}$ seconds).

This is a good choice because the modulated signal is composed of values spaced apart by $\frac{T_c}{2}$ seconds. By considering a sampling period of $\frac{T_c}{4}$ seconds we can sample the signal simply by considering two equal samples for each symbol in the signal $s(t)$.

Therefore, if for example we start from $s(t) = [-1, 1, -1 ...]$, after the process we are going to obtain $s(t) = [-1, -1, 1, 1, -1, -1 ...]$. Since, after the modulation, the length of the signal was 654720, after this process the length will be $654720\cdot2 = 1309440$ values.
Alternatively, we can also consider a sampling frequency $F_s = 8.184$ MHz (that refers to a sampling period $T_s = \frac{T_c}{8}$). In principle we can consider also bigger sampling frequencies, but energy and power consumption considerations must be also taken into account. Therefore we are going to use a sampling frequency $F_s = 4.092$ MHz (that correspond to $T_s = 2.4438\cdot10^{-7}$ seconds).


In [ ]:
#now we do the sampling of our signal
modulated_sampled, t_sampled = sampling(modulated, settings['fSampling'])

## POWER OF THE SIGNAL 

In satellite communications, the modulated signal has a power that is too low to reach the receiver. For this reason, at the ground station, a high power amplifier is utilized. It is a device that takes as input a signal, and produces a signal with a bigger amplitude, in order to increase its power.
Therefore, to simulate its action, we just need to amplify our sampled signal, that at the moment varies between 1 and -1.

If we want to obtain a transmitting power $P_t$, it's sufficient to set the amplitude of the signal as $\sqrt{P_t}$. This is because the power can be computed as:
$P_t = \dfrac{1}{N} \sum_{n=0}^{N-1}\,s^2\,[n]$, where $s$ is the signal we are considering.

In [ ]:
#COMPUTATION OF THE POWER OF THE SIGNAL
#At the moment the power of the signal is unitary, because each sample is either 1 or -1
#Therefore when we take the square of a sample we get 1, and if we sum them we get the length of the signal

signal_to_tx = hpa(modulated_sampled, t_sampled, settings['transmittedPower'])

## PROPAGATION THROUGH THE CHANNEL AND RECEIVED POWER

In order to simulate the propagation through the channel we need to consider the fact that the transmitted signal, with a transmitted power $P_t$, will be highly attenuated. The formula to compute the received power is the following:

$P_r = P_t - P_L + G_t + G_r$  (dB)

where $P_t$ is the transmitted power, $P_L$ is the path loss, $G_t$ is the gain of the transmitting antenna and $G_r$ is the gain of the received antenna. The path loss is provided by the orbital propagator group (as well as the other observables). 
We suppose we have a certain antenna, that can trasmit a signal whose power is set in the settings.ini file. From this value, with the formula described above, we can compute the received power. The target SNR to achieve at the receiver is also parameter of the settings.ini file, as well as the antenna gains. 

We simulate the signal at the receiver by introducing a time delay due to the propagation through the channel, and we multiply the original modulated signal (the one we had at the transmitter before the amplification) by $\sqrt{P_r}$, in order to simulate the amplitude variation caused by the attenuation. At this point we will compute the I/Q samples and we will add to them the noise, whose power $P_n$ is retrieved by inverting the SNR formula, since 

$P_n = \dfrac{P_r}{SNR}$

Those samples are going to be quantized and written in a binary file.

To take into account the doppler frequencies, the propagation delay and the FSPL we need to load the file provided by the orbit propagator group.

In [ ]:
#here we load the file provided by the orbit propagator group
filename = "OrbitalDATA.txt"
orbit = pd.read_table(filename, header=0, sep=' ')
orbit = orbit.dropna(axis=0)

#we also want to consider the Free Space Path Loss in dB, so we add a column to the dataframe
orbit['FSPL[dB]'] = orbit['FSPL'].apply(lin2dB)

#in order to estimate also the propagation delay, we can just divide the pseudorange (satellite-user distance)
#by the speed of light. Therefore we create another column in the dataframe
c = 299792458   #speed of light in vacuum in m/s
orbit['PropDelay[s]'] = orbit['Pseudorange[m]']/c

#adding a column for the indexes
orbit['Index'] = np.arange(0, orbit.shape[0], 1)
column_list = ["Index","Date", "Time", "Siminstant[s]", "Pseudorange[m]", "FSPL", "CodeDelay[chips]", 
               "DopplerShift[Hz]", "FSPL[dB]", "PropDelay[s]"]
orbit = orbit.reindex(columns=column_list)

orbit

In [ ]:
prop_delay = orbit['PropDelay[s]'][1]

dopplerFreqSamples = orbit['DopplerShift[Hz]'].dropna().to_numpy()
originalSamplingPeriod = 0.01
interpSamplingPeriod = 1 / settings['fSampling']
StartTime = 10
numInterpSamples=1309440 # with this value it might take 10/30 seconds

interpDopplerShift = GetDopplerShift(dopplerFreqSamples, originalSamplingPeriod, StartTime, numInterpSamples, interpSamplingPeriod)
interpDopplerShift2 = GetDopplerShift(dopplerFreqSamples, originalSamplingPeriod, prop_delay, numInterpSamples, interpSamplingPeriod)

print(interpDopplerShift)
print(interpDopplerShift2)

print(len(interpDopplerShift))  #length is okay
print(len(interpDopplerShift2))  #length is okay

In [ ]:
interpSamplingPeriod = 1 / settings['fSampling']
ciao = (np.asarray(range(0,numInterpSamples)))*interpSamplingPeriod
print(ciao)

InterpSampleTime = np.asarray([i+1 for i in ciao]) #shifts it 
print(InterpSampleTime)

As already said many times, the signal associated to a packet is composed of 1309440 samples, and each sample has a duration of $T_s = \frac{T_c}{4} = 2.4438 \cdot 10^{-7}$ seconds. Therefore the duration of the message associated to a packet is equal to $1309440 \cdot T_s = 0.32$ seconds. In the orbit dataframe the sampling period to compute the values of the observables is 10 ms, therefore 0.32 seconds (320 ms) corresponds to the first 320/10 = 32 rows of the dataframe. 
In this short period of time the free space path loss and the propagation delay change in a trascurable way, as shown in the following cell.

In [ ]:
print("FSPL of the first row: ", orbit['FSPL[dB]'][1])
print("FSPL of the 32th row: ", orbit['FSPL[dB]'][32])
print("Difference: ", np.abs(orbit['FSPL[dB]'][1] - orbit['FSPL[dB]'][32]))

print("\nPseudorange of the first row: ", orbit['PropDelay[s]'][1])
print("Pseudorange of the 32th row: ", orbit['PropDelay[s]'][32])
print("Difference: ", np.abs(orbit['PropDelay[s]'][1] - orbit['PropDelay[s]'][32]))

Since the difference is negligible, in order to simplify the simulation we are going to consider a constant propagation delay and a constant FSPL for the transmission of each packet. Another simplification that we consider is the following: since the packets are going to be transmitted one after the other, we can consider the propagation delay only once, and it will be the delay we observe when we start the transmission.

### RECEIVED SIGNAL

The received signal will be characterized by a multiplicative factor given by $\sqrt{P_r}$ and a time vector that has a length equal to 1309440, starts at the propagation delay $\tau_p$ and has one sample every $T_s = \frac{T_c}{4}$ seconds, as well as the transmitted signal. Therefore, for the signal referred to the first packet:

In [ ]:


# so now we have a function that returns the time vector at the receiver (with prop.delay considered only once) 
# corresponding at each packet

#INTERPOLATION OF FSPL NEEDED! AND CHECK IF THE VALUE RETURNED BY THE INTERPOLATION IS LINEAR OR DB
#array contains the interpolation of the FSPL (so one every Ts = Tc / 4 seconds), Pt[W], Gt(dB), Gr(dB), 
def return_amplitude(array_fspl, index, Pt, Gt, Gr):
    FSPL = array_fspl[index*1309440]  #check if array_fspl is linear or dB
    Pr_dB = lin2dB(Pt) + settings[Gt] + settings[Gr] - FSPL
    Pr = lin2dB(Pr_dB)
    return np.sqrt(Pr)


#for the doppler frequencies it is a little more complicated, because we need to select the first doppler frequency
#in correspondance of the propagation delay.
# def return_doppler(begin, array, )

In [ ]:
#received power (if we suppose we start to transmit at the time corresponding at the first line of the dataframe)

Pr_dB = lin2dB(settings['transmittedPower']) + settings['transmitterGain'] + settings['receiverGain'] - orbit['FSPL[dB]'][1]
Pr = dB2lin(Pr_dB)

#as propagation delay we consider the value of the first line of the dataframe
prop_delay = orbit['PropDelay[s]'][1]

received_signal = np.sqrt(Pr) * modulated_sampled 

print("Received signal: ", received_signal)

#here we compute the total time vector that accounts only once for the propagation delay
total_time_vector = np.arange(prop_delay, num_packets*1309440*(1/settings['fSampling']) + prop_delay, 1/settings['fSampling'])

#to compute the time vector of each packet we use the function described above. For the first packet (index = 0):
time_vector = return_time_vector(total_time_vector, 0)
# print(time_vector)

#so now we have the received signal (without the noise, that will be added to the IQ samples) and the correspondign time vector

xticks = time_vector[0:10]
fig = plt.figure(figsize=(15,5))
ax = fig.add_subplot(1,1,1)
ax.step(time_vector[0:10], received_signal[0:10], where='post', lw=3)
ax.set_xticks(xticks)
ax.set_yticks([-np.sqrt(Pr), 0, np.sqrt(Pr)])
ax.set_title('Received signal', fontsize=15)
ax.set_xlabel('Time [s]', fontsize=15)
ax.set_ylabel('Amplitude', fontsize=15)
ax.grid()


## I / Q SAMPLES AT THE RECEIVER   [TO COMPLETE]

The received signal has been generated, and we can call it $s(t)$. The following step consists in generating the I/Q samples, that are simply obtained by multiplying the signal $s(t)$ by a cosine and a sine function:

$x_I(t) = s(t)\,\cos\,(2\,\pi\,f'\,t)$

$x_Q(t) = s(t)\,\sin\,(2\,\pi\,f'\,t)$

where $f'$ is a vector that contains all the Doppler frequencies computed during the process. This is because the I/Q samples are supposed to be generated in baseband, therefore there is no carrier frequency to consider in this case.

To generate the samples, we need of course to sample the signals $x_I(t)$ and $x_Q(t)$ and the sampling frequencies used in these cases are in the order of 4 or 5 MHz. A good choice might be to consider a sampling frequency $F_s = 4.092$ MHz (with a sampling period $T_s = \frac{1}{F_s} = 2.4438\,\cdot10^{-7} = \frac{T_c}{4}$ seconds).

This is a good choice because the signal $s\,(t)$ is composed of values spaced apart by $\frac{T_c}{2}$ seconds. By considering a sampling period of $\frac{T_c}{4}$ seconds we can sample the signal simply by considering two equal samples for each symbol in the signal $s(t)$.

Therefore, if for example we start from $s(t) = [-1, 1, -1 ...]$, after the process we are going to obtain $s(t) = [-1, -1, 1, 1, -1, -1 ...]$. Since, after the modulation, the length of $s(t)$ was 654720, after this process the length of $s(t)$ will be $654720\cdot2 = 1309440$ values.
Alternatively, we can also consider a sampling frequency $F_s = 8.184$ MHz (that refers to a sampling period $T_s = \frac{T_c}{8}$). In principle we can consider also bigger sampling frequencies, but energy and power consumption considerations must be also taken into account. Therefore we are going to use a sampling frequency $F_s = 4.092$ MHz (that correspond to $T_s = 2.4438\cdot10^{-7}$ seconds).

Regarding the Doppler frequencies, they are calculated by the orbit propagator group, and each value is referred to a new update of the satellite position. This is done with a sampling period that is way bigger than $\frac{T_c}{4}$, because the orbit parameters are typically updated less frequently. Therefore, in order to obtain a vector $f'$ with 1309440 values (to allow the multiplication of the cosine with $s(t))$, we need to interpolate the data provided by the other group.



### INTERPOLATION OF DOPPLER FREQUENCIES

In [ ]:
###############################################
#                                             #
#  CODE FOR INTERPOLATION MUST BE ADDED HERE  #
#                                             #
###############################################

#For the moment, since we don't have the data from the other group, we can just simulate the vector f'
final_length = 1309440
f_prime = np.zeros(final_length)
f_max = 3000

f_prime[0] = np.random.uniform(-f_max, f_max, 1)

#with the following for loop we generate doppler frequencies between -3 KHz and 3 KHz. The absolute value of 
#the differential doppler frequency between two consecutive symbols is not greater than 0.3 Hz (from Mattia).
for i in range(1,final_length):
        f_prime[i] = np.random.uniform(f_prime[i-1], f_prime[i-1] + 0.3, 1)
    
print(f_prime[0:20])
print("\nLength of the vector f':", len(f_prime))

In [ ]:
#so now we have the vector of Doppler frequencies, therefore we need to sample the signal s(t) (called "modulated"
#here) and to create the time vector t.
#As already said, each symbol of the vector s(t) has a duration of Tc / 2. Therefore if we use Ts = Tc / 4 we
#just need to repeat each symbol of s(t) two times. This can be easily done with the function np.repeat()


# ALREADY DONE AFTER THE MODULATION


# Fs = 4.092e06    #sampling frequency is 4.092 MHz
# Ts = 1 / Fs      #sampling period, equal to Tc/4

# #creation of the sampled signal
# modulated_sampled = np.repeat(modulated, 2)
# print("Before sampling:", modulated[0:9])
# print("After sampling", modulated_sampled[0:18])
# print("Length after sampling:", len(modulated_sampled))

# #creation of the time vector
# t_sampled = np.arange(0, final_length * Ts, Ts)   #1309440 values spaced apart by Tc / 4 seconds each
# print("\nTime vector:", t_sampled)
# print("Time vector length:", len(t_sampled))

Now all the vectors have the right length, and we can obtain $x_I(t)$ and $x_Q(t)$, and we can also add the noise using the awgn function defined here.

In [ ]:
t_sampled = np.arange(0, final_length * Ts, Ts) #1309440 values spaced apart by Tc / 4 seconds each 
                                                #last value is 0.32 seconds
t_sampled2 = np.arange(t_sampled[-1], 2*final_length * Ts, Ts)
print("\n", t_sampled2)
print(len(t_sampled2))
# print(final_length)

In [ ]:
I = modulated_sampled*np.cos(2*pi*f_prime*t_sampled) 
print("\nI:", I[0:20])

#addition of noise 
I_noise = awgn(I, -30, True)
print("\n\nAfter noise addition on I:\n", I_noise[0:20])

Q = modulated_sampled*np.sin(2*pi*f_prime*t_sampled)
print("\nQ:", Q[0:20])

#addition of noise (noise power = -50 dB): THIS VALUE MUST BE CHECKED, NO IDEA ABOUT NOISE POWER TYPICAL VALUES
Q_noise = awgn(Q, -50, True)
print("\n\nAfter noise addition on Q:\n", Q_noise[0:20])

#some plots (first N samples)
N = 30

fig = plt.figure(figsize=(15,5))
ax = fig.add_subplot(1,1,1)
xticks = t_sampled[0:N]
ax.stem(t_sampled[0:N], I[0:N])
ax.set_xticks(xticks)
plt.xticks(rotation=45)
ax.set_title("I samples before noise addition", fontsize=15)
ax.set_xlabel("Time [s]", fontsize=15)
ax.set_ylabel("I samples", fontsize=15)
ax.grid()

fig = plt.figure(figsize=(15,5))
ax = fig.add_subplot(1,1,1)
xticks = t_sampled[0:N]
ax.stem(t_sampled[0:N], I_noise[0:N])
ax.set_xticks(xticks)
plt.xticks(rotation=45)
ax.set_title("I samples after noise addition", fontsize=15)
ax.set_xlabel("Time [s]", fontsize=15)
ax.set_ylabel("I samples", fontsize=15)
ax.grid()

## QUANTIZATION

Before writing the I/Q samples of the modulated signal in a binary file, we need to quantize them. In order to do that, we consider a uniform quantization with 8 bits, therefore we divide the interval of amplitudes between -1 and 1 in $2^8$ = 256 levels, equally spaced apart between each other.
Each sample will be substituted by the value of the level that is closest to it.
The function **quantization** takes as input the array to quantize, the upper and lower bounds of the interval from which the levels are drawn, the number of bits for the quantization and a flag that, if true, prints the set of levels of the quantization.

In [ ]:
I_quantized = quantization(I_noise, 1, -1, 8, True)
print("\nI_noise: ", I_noise[0:10] )
print("\nI_quantized", I_quantized[0:10])        
    
Q_quantized = quantization(Q_noise, 1, -1, 8, False)
print("\nQ_noise: ", Q_noise[0:10] )
print("\nQ_quantized", Q_quantized[0:10])        

# print(len(Q_quantized))
# print(len(Q_noise))